In [1]:
from __future__ import print_function

import collections

# Import Python wrapper for or-tools CP-SAT solver.
from ortools.sat.python import cp_model

In [2]:
clever = False
start_test = 0
end_test = 15

In [3]:
class Solution():
    def __init__(self):
        self.time = []

In [4]:
class Task():
    def __init__(self):
        self.name = None
        self.machine_num = None
        self.job_num = None
        self.task_num  = None
        self.start = None
        self.end = None

In [5]:
global_sol = Solution()

In [6]:
def MinimalJobshopSat(jobs_data):
    """Minimal jobshop problem."""
    # Create the model.
    model = cp_model.CpModel()

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    jobs_count = len(jobs_data)
    all_jobs = range(jobs_count)

    # Compute horizon.
    horizon = sum(task[1] for job in jobs_data for task in job)

    task_type = collections.namedtuple('task_type', 'start end interval')
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index')

    # Create jobs.
    all_tasks = {}
    for job in all_jobs:
        for task_id, task in enumerate(jobs_data[job]):
            start_var = model.NewIntVar(0, horizon,
                                        'start_%i_%i' % (job, task_id))
            duration = task[1]
            end_var = model.NewIntVar(0, horizon, 'end_%i_%i' % (job, task_id))
            interval_var = model.NewIntervalVar(
                start_var, duration, end_var, 'interval_%i_%i' % (job, task_id))
            all_tasks[job, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        intervals = []
        for job in all_jobs:
            for task_id, task in enumerate(jobs_data[job]):
                if task[0] == machine:
                    intervals.append(all_tasks[job, task_id].interval)
        model.AddNoOverlap(intervals)

    # Add precedence contraints.
    for job in all_jobs:
        for task_id in range(0, len(jobs_data[job]) - 1):
            model.Add(all_tasks[job, task_id +
                                1].start >= all_tasks[job, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(
        obj_var,
        [all_tasks[(job, len(jobs_data[job]) - 1)].end for job in all_jobs])
    model.Minimize(obj_var)

    # Solve model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        # Print out makespan.
        print('Optimal Schedule Length: %i' % solver.ObjectiveValue())
        print()

        # Create one list of assigned tasks per machine.
        assigned_jobs = [[] for _ in all_machines]
        for job in all_jobs:
            for task_id, task in enumerate(jobs_data[job]):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.Value(all_tasks[job, task_id].start),
                        job=job,
                        index=task_id))

        disp_col_width = 10
        sol_line = ''
        sol_line_tasks = ''

        print('Optimal Schedule', '\n')

        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line += 'Machine ' + str(machine) + ': '
            sol_line_tasks += 'Machine ' + str(machine) + ': '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_%i' % (assigned_task.job, assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += name + ' ' * (disp_col_width - len(name))
                start = assigned_task.start
                duration = jobs_data[assigned_task.job][assigned_task.index][1]

                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                #sol_line += sol_tmp + ' ' * (disp_col_width - len(sol_tmp))
                sol_line += sol_tmp + ' '

                
            sol_line += '\n'
            sol_line_tasks += '\n'

        print(sol_line_tasks)
        print('Task Time Intervals\n')
        print(sol_line)
        global_sol.sol_line_tasks = sol_line_tasks.split()
        global_sol.sol_line = sol_line.split()
        global_sol.time.append(solver.ObjectiveValue())



In [7]:
jobs_data = [  # task = (machine_id, processing_time).
    [(0, 3), (1, 2), (2, 2)],  # Job0
    [(0, 2), (2, 1), (1, 4)],  # Job1
    [(1, 4)]  # Job2
]
#MinimalJobshopSat(jobs_data)

In [8]:
global_obtained_sol = []

In [9]:
def obtain_sol_line_tasks():
    cur_machine_num = None
    cur_task = None
    for k in range(len(global_sol.sol_line_tasks)):
        i = global_sol.sol_line_tasks[k]
        j = global_sol.sol_line[k]
        if ':' in i:
            cur_machine_num = i[:-1]
        elif 'job' in i:
            cur_task = Task()
            job_info = i.split('_')
            cur_task.task_num = int(job_info[2])
            cur_task.job_num = int(job_info[1])
            cur_task.machine_num = int(cur_machine_num)
            
            time_info = global_sol.sol_line[k][1:-1].split(',')
            start, end = int(time_info[0]), int(time_info[1])
            cur_task.start = start
            cur_task.end = end
            global_obtained_sol.append(cur_task)


In [10]:
#obtain_sol_line_tasks()

In [11]:
def check(sol):
    for task in sol:
        print('job {j}, task {t}, machine {m}, start {s}, end {e}'.format(
                    t=str(task.task_num),
                    m=str(task.machine_num),
                    j=str(task.job_num),
                    s=str(task.start),
                    e=str(task.end)
        ))
    

In [12]:
#check(global_obtained_sol)

In [13]:
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
plotly.tools.set_credentials_file(username='sko_hack', api_key='kQEUP4Lw2ebhAjKoU0sJ')

In [14]:
def create_data_frame(start, end):
    import datetime
    the_start = datetime.datetime(2019, 2, 24)
    s = the_start + datetime.timedelta(seconds=start*60)
    e = the_start + datetime.timedelta(seconds=end*60)
    
    start = str(s)
    end = str(e)
    
    return (start, end)
    
    
    

In [15]:
'''
df = []
for task in global_obtained_sol:
    data = create_data_frame(task.start, task.end)
    df.append(dict(
        Task=str(task.machine_num),
        Start=data[0],
        Finish=data[1],
        Resource=task.job_num
    ))

colors = {
    0: 'rgb(0, 0, 0)',
    1: 'rgb(128, 0, 128)',
    2: 'rgb(255, 0, 0)',
    3: 'rgb(0, 255, 0)',
    4: 'rgb(0, 0, 255)',
    5: 'rgb(255, 255, 0)',
    6: 'rgb(0, 255, 255)',
    7: 'rgb(255, 0, 255)',
    8: 'rgb(192, 192, 192)',
    9: 'rgb(128, 128, 128)',
    10: 'rgb(128, 0, 0)'
}


fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)
py.iplot(fig, filename='gantt-group-tasks-together', world_readable=True)
'''


"\ndf = []\nfor task in global_obtained_sol:\n    data = create_data_frame(task.start, task.end)\n    df.append(dict(\n        Task=str(task.machine_num),\n        Start=data[0],\n        Finish=data[1],\n        Resource=task.job_num\n    ))\n\ncolors = {\n    0: 'rgb(0, 0, 0)',\n    1: 'rgb(128, 0, 128)',\n    2: 'rgb(255, 0, 0)',\n    3: 'rgb(0, 255, 0)',\n    4: 'rgb(0, 0, 255)',\n    5: 'rgb(255, 255, 0)',\n    6: 'rgb(0, 255, 255)',\n    7: 'rgb(255, 0, 255)',\n    8: 'rgb(192, 192, 192)',\n    9: 'rgb(128, 128, 128)',\n    10: 'rgb(128, 0, 0)'\n}\n\n\nfig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)\npy.iplot(fig, filename='gantt-group-tasks-together', world_readable=True)\n"

In [16]:
'''
df = []
for task in global_obtained_sol:
    data = create_data_frame(task.start, task.end)
    df.append(dict(
        Task=str(task.job_num),
        Start=data[0],
        Finish=data[1],
        Resource=task.machine_num
    ))

colors = {
    0: 'rgb(0, 0, 0)',
    1: 'rgb(128, 0, 128)',
    2: 'rgb(255, 0, 0)',
    3: 'rgb(0, 255, 0)',
    4: 'rgb(0, 0, 255)',
    5: 'rgb(255, 255, 0)',
    6: 'rgb(0, 255, 255)',
    7: 'rgb(255, 0, 255)',
    8: 'rgb(192, 192, 192)',
    9: 'rgb(128, 128, 128)',
    10: 'rgb(128, 0, 0)'
}


fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)
py.iplot(fig, filename='gantt-group-tasks-together', world_readable=True)
'''

"\ndf = []\nfor task in global_obtained_sol:\n    data = create_data_frame(task.start, task.end)\n    df.append(dict(\n        Task=str(task.job_num),\n        Start=data[0],\n        Finish=data[1],\n        Resource=task.machine_num\n    ))\n\ncolors = {\n    0: 'rgb(0, 0, 0)',\n    1: 'rgb(128, 0, 128)',\n    2: 'rgb(255, 0, 0)',\n    3: 'rgb(0, 255, 0)',\n    4: 'rgb(0, 0, 255)',\n    5: 'rgb(255, 255, 0)',\n    6: 'rgb(0, 255, 255)',\n    7: 'rgb(255, 0, 255)',\n    8: 'rgb(192, 192, 192)',\n    9: 'rgb(128, 128, 128)',\n    10: 'rgb(128, 0, 0)'\n}\n\n\nfig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)\npy.iplot(fig, filename='gantt-group-tasks-together', world_readable=True)\n"

In [17]:
machine_table = {}
machine_table['Пила'] = [0]
machine_table['Плазменная'] = [1]
machine_table['Токарный'] = [2, 3]
machine_table['Фрезерный'] = [4, 5]
machine_table['Сверление'] = [6, 7]
machine_table['Сварка'] = [8]
machine_table['Окраска'] = [9]

In [18]:
# making good legend
machine_name = {}
machine_name[0] = 'Пила'
machine_name[1] = 'Плазменная'
machine_name[2] = 'Токарный_0'
machine_name[3] = 'Токарный_1'
machine_name[4] = 'Фрезерный_0'
machine_name[5] = 'Фрезерный_1'
machine_name[6] = 'Сверление_0'
machine_name[7] = 'Сверление_1'
machine_name[8] = 'Сварка'
machine_name[9] = 'Окраска'


In [19]:
def obtain_path(path_name):
    paths = []
    for name in path_name:
        new_paths = []
        if len(paths) == 0:
            for machine in machine_table[name]:
                new_paths.append([machine])
        else:
            for p in paths:
                for machine in machine_table[name]:
                    new_paths.append(p + [machine])
        paths = new_paths
    return paths

In [20]:
obtain_path(['Пила', 'Плазменная'])

[[0, 1]]

In [21]:
import pandas as pd
import numpy as np
file_name = 'Khakaton.xlsx'
products = pd.read_excel(open('Khakaton.xlsx', 'rb'), sheet_name=u'Перечень и состав изделий')
prods_num = pd.read_excel(open('Khakaton.xlsx', 'rb'), sheet_name=u'Необходимое кол-во изделий')
equipment = pd.read_excel(open('Khakaton.xlsx', 'rb'), sheet_name=u'Оборудование')
lines = pd.read_excel(open('Khakaton.xlsx', 'rb'), sheet_name=u'маршрут изготовления', header=2)                  
process_time = pd.read_excel(open('Khakaton.xlsx', 'rb'), sheet_name=u'нормы времени (мин.)')
adjustment_time = pd.read_excel(open('Khakaton.xlsx', 'rb'), sheet_name=u'Время на наладку оборудования')
submisson = pd.read_excel(open('Khakaton.xlsx', 'rb'), sheet_name=u'мой вспомогательный')

In [22]:
all_the = [products, prods_num, equipment, lines, process_time, adjustment_time]

In [23]:
for i in all_the:
    print(i.shape)

(72, 2)
(103, 3)
(10, 3)
(72, 7)
(72, 9)
(72, 9)


In [24]:
products.head()

,N,nomenclature
0,1.0,Переходник 1
1,1.1,Муфта А1
2,1.2,Муфта А2
3,2.0,Переходник 2
4,2.1,Муфта Б1


In [25]:
prods_num.head()

,N,nomenclature,number
0,1.0,Переходник 1,10
1,1.1,Муфта А1,10
2,1.2,Муфта А2,10
3,2.0,Переходник 2,10
4,2.1,Муфта Б1,10


In [26]:
equipment.head()

,N,machine,type
0,1,Станок №1,Пила
1,2,Станок №2,Плазменная резка
2,3,Станок №3,Токарный 1
3,4,Станок №4,Токарный 2
4,5,Станок №5,Фрезерный 1


In [27]:
lines.head()

,N,nomenclature,operation1,operation2,operation3,operation4,operation5
0,1.0,Переходник 1,Сварка,Окраска,NaN,NaN,NaN
1,1.1,Муфта А1,Пила,Токарный,Фрезерный,Сверление,NaN
2,1.2,Муфта А2,Пила,Токарный,Фрезерный,Сверление,NaN
3,2.0,Переходник 2,Сварка,Окраска,NaN,NaN,NaN
4,2.1,Муфта Б1,Пила,Токарный,Фрезерный,Сверление,NaN


In [28]:
process_time.head()

,N,nomenclature,saw,plazma,turning,milling,drilling,welding,painting
0,1.0,Переходник 1,NaN,NaN,NaN,NaN,NaN,25.0,10.0
1,1.1,Муфта А1,1.0,NaN,3.0,5.0,5.0,NaN,NaN
2,1.2,Муфта А2,1.0,NaN,3.0,5.0,5.0,NaN,NaN
3,2.0,Переходник 2,NaN,NaN,NaN,NaN,NaN,35.0,10.0
4,2.1,Муфта Б1,1.0,NaN,3.0,5.0,1.0,NaN,NaN


In [29]:
adjustment_time.head()

,N,nomenclature,saw,plazma,turning,milling,drilling,welding,painting
0,1.0,Переходник 1,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,1.1,Муфта А1,5.0,NaN,10.0,15.0,1.0,NaN,NaN
2,1.2,Муфта А2,5.0,NaN,10.0,15.0,1.0,NaN,NaN
3,2.0,Переходник 2,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,2.1,Муфта Б1,5.0,NaN,10.0,15.0,1.0,NaN,NaN


In [30]:
N = 72 # overall number of elements
elements_parts = [products, lines, process_time, adjustment_time]

In [31]:
products_np = products.values
lines_np = lines.values
process_time_np = process_time.values
adjustment_time_np = adjustment_time.values
prods_num_np = prods_num.values


In [32]:
elements = []

In [33]:
ht = {}
ht['Пила'] = 2
ht['Плазменная'] = 3
ht['Токарный'] = 4
ht['Фрезерный'] = 5
ht['Сверление'] = 6
ht['Сварка'] = 7
ht['Окраска'] = 8

In [34]:
class Detail():
    def __init__(self, name, path, proc_time, adj_time):
        self.name = name    
        self.path = path   
        self.proc_time = proc_time
        self.adj_time = adj_time

In [35]:
details = []

In [36]:
for i in range(N):
    number = i
    name = products_np[i][1]
    path = []
    for path_part in lines_np[i][2:]:
        if path_part == path_part:
            path.append(path_part)
    proc_time = []
    adj_time = []
    
    for _ in path:
        proc_time.append(process_time_np[i][ht[_]])
        adj_time.append(adjustment_time_np[i][ht[_]])
        
    new_element = Detail(name, path, proc_time, adj_time)
    details.append(new_element)
    
        
    

In [37]:
d = details[25]
print('name = {} path = {} proc_time = {} adj_time = {}'.format(d.name, d.path, d.proc_time, d.adj_time))

name = Втулка 5Д path = ['Пила', 'Токарный', 'Фрезерный'] proc_time = [2.0, 5.0, 5.0] adj_time = [5.0, 5.0, 5.0]


In [38]:
name_to_id = {}

In [39]:
for idx, i in enumerate(details):
    name_to_id[i.name] = idx

In [40]:
# MAIN PART OF THE ALGORITM !!!

In [41]:
order = []
j_names = []

In [42]:
N_correct = 50

In [43]:
for i in range(N_correct):
    order.append((name_to_id[prods_num_np[i][1]], prods_num_np[i][2]))
    j_names.append(prods_num_np[i][1])
    
    

In [44]:
# Simple Case
full_order = list(order)
full_j_names = list(j_names)
banned = [0, 3, 6, 13, 20] 
for i in range(len(banned) - 1, -1, -1):
    del j_names[banned[i]]
    del order[banned[i]]

In [45]:
# The most important part

In [46]:
tasks =  []

In [47]:
# Task is a pair of 2 integers: 
# Number of the machine
# The length of the task
# Job is an ordered tuple of tasks

In [48]:
def create_job(task_id, num_of_tasks): # 
    task = details[task_id]
    path = task.path
    proc_time = task.proc_time
    adj_time = task.adj_time
    name = task.name
    
    job = []
    
    paths_to_leaf = obtain_path(path)
    
    # Choosing path
    import random
    chosen_path = paths_to_leaf[random.randint(0, len(paths_to_leaf) - 1)]
    
    
    for i in range(len(path)):
        adjustment = (chosen_path[i], int(adj_time[i]))
        task = (chosen_path[i], int(proc_time[i] * num_of_tasks))
        job.append(adjustment)
        job.append(task)
    return job
    
    
    

In [49]:
def create_job_clever(task_id, num_of_tasks): 
    task = details[task_id]
    path = task.path
    proc_time = task.proc_time
    adj_time = task.adj_time
    name = task.name
    
    job = []
    
    paths_to_leaf = obtain_path(path)
    
    if len(paths_to_leaf) == 1:
        return (1, create_job(task_id, num_of_tasks))
    
    # Choosing path_0
    import random
    chosen_path_0 = paths_to_leaf[random.randint(0, len(paths_to_leaf) - 1)]
    
    total_proc_time = 0
    total_adj_time = 0
    for i in range(len(path)):
        total_proc_time += proc_time[i] * num_of_tasks
        total_adj_time += adj_time[i]
    
    eps = 0.1
    if total_adj_time > eps * total_proc_time:
        for i in range(len(path)):
            adjustment = (chosen_path_0[i], int(adj_time[i]))
            task = (chosen_path_0[i], int(proc_time[i] * num_of_tasks))
            job.append(adjustment)
            job.append(task)
        return (1, job)
    else:
        chosen_path_1 = paths_to_leaf[random.randint(0, len(paths_to_leaf) - 1)]
        job_0 = []
        job_1 = []
        for i in range(len(path)):
            adjustment_0 = (chosen_path_0[i], int(adj_time[i]))
            adjustment_1 = (chosen_path_1[i], int(adj_time[i]))
            
            task_0 = (chosen_path_0[i], int(proc_time[i] * num_of_tasks // 2))
            job_0.append(adjustment_0)
            job_0.append(task_0)
            
            task_1 = (chosen_path_1[i], int(proc_time[i] * (num_of_tasks - num_of_tasks // 2)))
            job_1.append(adjustment_1)
            job_1.append(task_1)

        return (2, job_0, job_1)
            
        

In [50]:
mvp = []

In [51]:
max_len = len(order)
print(max_len)

45


In [52]:
def clever_algo(start, end):
    max_len = len(order)

    for i in range(start, end):
        #task = create_job(order[i][0], order[i][1])

        n_task = create_job_clever(order[i][0], order[i][1])
        if n_task[0] == 1:
            mvp.append(n_task[1])
        elif n_task[0] == 2:
            mvp.append(n_task[1])
            mvp.append(n_task[2])
        #mvp.append(task)

In [53]:
def stupid_algo(start, end):
    max_len = len(order)

    for i in range(start, end):
        task = create_job(order[i][0], order[i][1])
        mvp.append(task)
    

In [54]:
if clever:
    clever_algo(start_test, end_test)
else:
    stupid_algo(start_test, end_test)

In [55]:
print(mvp)

[[(0, 5), (0, 10), (2, 10), (2, 30), (4, 15), (4, 50), (6, 1), (6, 50)], [(0, 5), (0, 10), (3, 10), (3, 30), (5, 15), (5, 50), (7, 1), (7, 50)], [(0, 5), (0, 10), (3, 10), (3, 30), (4, 15), (4, 50), (7, 1), (7, 10)], [(0, 5), (0, 10), (2, 10), (2, 20), (4, 15), (4, 50), (7, 1), (7, 10)], [(5, 25), (5, 240), (2, 10), (2, 150), (6, 10), (6, 60)], [(0, 5), (0, 30), (2, 10), (2, 210)], [(0, 5), (0, 60), (2, 10), (2, 240), (6, 1), (6, 150)], [(0, 5), (0, 60), (2, 10), (2, 180)], [(0, 5), (0, 60), (2, 10), (2, 150), (5, 5), (5, 270)], [(0, 5), (0, 60), (3, 10), (3, 60), (5, 5), (5, 360)], [(5, 25), (5, 70), (2, 10), (2, 15), (7, 10), (7, 20)], [(0, 5), (0, 5), (3, 10), (3, 35)], [(0, 5), (0, 5), (3, 10), (3, 120), (7, 1), (7, 20)], [(0, 5), (0, 5), (2, 10), (2, 170)], [(0, 5), (0, 5), (3, 10), (3, 15), (4, 5), (4, 5)]]


In [56]:
global_sol = Solution()

In [57]:
MinimalJobshopSat(mvp)

Optimal Schedule Length: 1265

Optimal Schedule 

Machine 0: job_13_0  job_13_1  job_1_0   job_3_0   job_1_1   job_0_0   job_0_1   job_2_0   job_3_1   job_5_0   job_5_1   job_6_0   job_6_1   job_7_0   job_7_1   job_8_0   job_8_1   job_9_0   job_9_1   job_11_0  job_11_1  job_12_0  job_12_1  job_2_1   job_14_0  job_14_1  
Machine 1: 
Machine 2: job_13_2  job_13_3  job_0_2   job_0_3   job_3_2   job_3_3   job_5_2   job_10_2  job_10_3  job_6_2   job_7_2   job_8_2   job_6_3   job_4_2   job_8_3   job_7_3   job_4_3   job_5_3   
Machine 3: job_1_2   job_1_3   job_9_2   job_9_3   job_11_2  job_11_3  job_12_2  job_12_3  job_14_2  job_14_3  job_2_2   job_2_3   
Machine 4: job_3_4   job_3_5   job_0_4   job_0_5   job_14_4  job_14_5  job_2_4   job_2_5   
Machine 5: job_4_0   job_10_0  job_1_4   job_10_1  job_4_1   job_9_4   job_9_5   job_1_5   job_8_4   job_8_5   
Machine 6: job_0_6   job_0_7   job_6_4   job_6_5   job_4_4   job_4_5   
Machine 7: job_3_6   job_3_7   job_12_4  job_12_5  job_2_6   job_2

In [58]:
global_obtained_sol = []
obtain_sol_line_tasks()

In [59]:
# First level

df = []
for task in global_obtained_sol:
    data = create_data_frame(task.start, task.end)
    df.append(dict(
        Task=str(j_names[task.job_num]),
        Start=data[0],
        Finish=data[1],
        Resource=machine_name[task.machine_num]
    ))

colors = {
    'Пила': 'rgb(0, 0, 0)',
    'Плазменная': 'rgb(128, 0, 128)',
    'Токарный_0': 'rgb(255, 0, 0)',
    'Токарный_1': 'rgb(0, 255, 0)',
    'Фрезерный_0': 'rgb(0, 0, 255)',
    'Фрезерный_1': 'rgb(255, 255, 0)',
    'Сверление_0': 'rgb(0, 255, 255)',
    'Сверление_1': 'rgb(255, 0, 255)',
    'Сварка': 'rgb(192, 192, 192)',
    'Окраска': 'rgb(128, 128, 128)',
}


fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)
py.iplot(fig, filename='gantt-group-tasks-together', world_readable=True)


High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sko_hack/0 or inside your plot.ly account where it is named 'gantt-group-tasks-together'


/home/genrikh/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [60]:
# Lets draw individual graphs for each machine

In [61]:
N = 2 # machine num
df = []
for task in global_obtained_sol:
    if task.machine_num == N:
        data = create_data_frame(task.start, task.end)
        df.append(dict(
            Task=str(j_names[task.job_num]),
            Start=data[0],
            Finish=data[1],
            Resource=machine_name[task.machine_num]
        ))

colors = {
    'Пила': 'rgb(0, 0, 0)',
    'Плазменная': 'rgb(128, 0, 128)',
    'Токарный_0': 'rgb(255, 0, 0)',
    'Токарный_1': 'rgb(0, 255, 0)',
    'Фрезерный_0': 'rgb(0, 0, 255)',
    'Фрезерный_1': 'rgb(255, 255, 0)',
    'Сверление_0': 'rgb(0, 255, 255)',
    'Сверление_1': 'rgb(255, 0, 255)',
    'Сварка': 'rgb(192, 192, 192)',
    'Окраска': 'rgb(128, 128, 128)',
}



fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)
py.iplot(fig, filename='gantt-group-tasks-together', world_readable=True)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sko_hack/0 or inside your plot.ly account where it is named 'gantt-group-tasks-together'


In [62]:
# First level time
final_time = global_sol.time[0]
global_sol.time[0]

1265.0

In [63]:
# Finally, lets create complex details

In [64]:
not_banned = [0, 3, 6, 13, 20]
order = full_order
j_names = full_j_names

for i in range(len(order) - 1, -1, -1):
    if i not in not_banned:
        del order[i]
        del j_names[i]
        
print(order)

[(0, 10), (3, 10), (6, 30), (13, 5), (20, 5)]


In [65]:
tasks = []
mvp = []
stupid_algo(0, len(order))

global_sol = Solution()

In [66]:
MinimalJobshopSat(mvp)

Optimal Schedule Length: 2850

Optimal Schedule 

Machine 0: 
Machine 1: 
Machine 2: 
Machine 3: 
Machine 4: 
Machine 5: 
Machine 6: 
Machine 7: 
Machine 8: job_0_0   job_0_1   job_1_0   job_2_0   job_3_0   job_4_0   job_1_1   job_2_1   job_3_1   job_4_1   
Machine 9: job_0_2   job_0_3   job_1_2   job_1_3   job_2_2   job_2_3   job_3_2   job_3_3   job_4_2   job_4_3   

Task Time Intervals

Machine 0: 
Machine 1: 
Machine 2: 
Machine 3: 
Machine 4: 
Machine 5: 
Machine 6: 
Machine 7: 
Machine 8: [0,0] [0,250] [0,0] [0,0] [0,0] [0,0] [250,600] [600,2250] [2250,2525] [2525,2800] 
Machine 9: [250,250] [250,350] [600,600] [600,700] [2250,2250] [2250,2550] [2550,2550] [2550,2600] [2800,2800] [2800,2850] 



In [67]:
global_obtained_sol = []
obtain_sol_line_tasks()

In [68]:
# Second level

df = []
for task in global_obtained_sol:
    data = create_data_frame(task.start, task.end)
    df.append(dict(
        Task=str(j_names[task.job_num]),
        Start=data[0],
        Finish=data[1],
        Resource=machine_name[task.machine_num]
    ))


colors = {
    'Пила': 'rgb(0, 0, 0)',
    'Плазменная': 'rgb(128, 0, 128)',
    'Токарный_0': 'rgb(255, 0, 0)',
    'Токарный_1': 'rgb(0, 255, 0)',
    'Фрезерный_0': 'rgb(0, 0, 255)',
    'Фрезерный_1': 'rgb(255, 255, 0)',
    'Сверление_0': 'rgb(0, 255, 255)',
    'Сверление_1': 'rgb(255, 0, 255)',
    'Сварка': 'rgb(192, 192, 192)',
    'Окраска': 'rgb(128, 128, 128)',
}


fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)
py.iplot(fig, filename='gantt-group-tasks-together', world_readable=True)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sko_hack/0 or inside your plot.ly account where it is named 'gantt-group-tasks-together'


In [69]:
print(global_sol.time[0])
#final_time += global_sol.time[0]

2850.0


In [70]:
# FINAL ANSWER
print('The overall time equals: {} mins'.format(str(final_time)))

The overall time equals: 1265.0 mins
